# Periodic Autoregressive (PAR(1)) modelling of quarterly inflows (Spain – gauging station 9087 Grisén)

This notebook implements a reproducible workflow to:

1) load gauging data exported from the Spanish *Anuario de Aforos* (MITECO) (SpreadsheetML/XML),
2) build hydrological-quarter inflows (hm³),
3) fit a periodic autoregressive model PAR(1),
4) generate synthetic equiprobable series,
5) compute Q75 using the Gringorten plotting position,
6) run basic normality checks on the transformed/standardized series,
7) export key outputs (tables/figures) to Excel and PNG.

Data source statement (as requested by MITECO): “© Ministerio para la Transición Ecológica y el Reto Demográfico”.


In [ ]:
from google.colab import drive
drive.mount("/content/drive")


In [ ]:
# Install dependencies (Colab usually has most of these)
!pip -q install pandas numpy scipy matplotlib openpyxl


In [ ]:
import os, pathlib

# Change this if you want a different working folder in your Drive:
BASE_DIR = "/content/drive/MyDrive/PAR_project"
RAW_DIR  = os.path.join(BASE_DIR, "data", "raw")
OUT_DIR  = os.path.join(BASE_DIR, "outputs")
TAB_DIR  = os.path.join(OUT_DIR, "tables")
FIG_DIR  = os.path.join(OUT_DIR, "figures")

for d in [RAW_DIR, TAB_DIR, FIG_DIR]:
    os.makedirs(d, exist_ok=True)

print("BASE_DIR:", BASE_DIR)


## 1. Input data

Upload the XML (SpreadsheetML) exported from the *Anuario de Aforos*.

If you prefer, you can also manually place the XML into `RAW_DIR` in your Drive.

In [ ]:
from google.colab import files
import shutil, os

uploaded = files.upload()  # upload the XML file
xml_files = [fn for fn in uploaded.keys() if fn.lower().endswith(".xml")]
if not xml_files:
    raise ValueError("No XML file uploaded. Please upload the SpreadsheetML/XML export.")

xml_name = xml_files[0]
xml_path = os.path.join(RAW_DIR, xml_name)
shutil.move(xml_name, xml_path)
print("Saved XML to:", xml_path)


In [1]:
# =========================================================
# BLOQUE II (Tema 3) - PAR(1) TRIMESTRAL (AÑO HIDROLÓGICO)
# Estación: 9087 Grisén (río Jalón) - CHE
# Entrada: XML (SpreadsheetML) exportado del Anuario de Aforos
# Salida: Serie trimestral (hm3) + ajuste PAR(1) por trimestre hidrológico
# =========================================================

# 0) IMPORTS
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import math

# ---------------------------------------------------------
# 1) MONTAR DRIVE (COLAB)
# ---------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')

# ---------------------------------------------------------
# 2) RUTA DEL XML (AJUSTA SI TU CARPETA ES DISTINTA)
#    Estructura recomendada:
#    /PAR_BloqueII/data_raw/Aforo_Estación_9087_Grisen.xml
# ---------------------------------------------------------
xml_path = "BASE_DIR"

# ---------------------------------------------------------
# 3) LEER XML TIPO "EXCEL XML" (SpreadsheetML)
#    Este XML suele tener: Estación | Año hidrológico | Día | Oct..Sep
#    Donde el año aparece como "1969-1970" (Oct-Sep).
# ---------------------------------------------------------
ns = {"ss": "urn:schemas-microsoft-com:office:spreadsheet"}
tree = ET.parse(xml_path)
root = tree.getroot()

ws = root.find("ss:Worksheet", ns)
table = ws.find("ss:Table", ns)
rows = table.findall("ss:Row", ns)

def row_values(row):
    """Extrae valores de una fila del SpreadsheetML."""
    vals = []
    for cell in row.findall("ss:Cell", ns):
        data = cell.find("ss:Data", ns)
        vals.append(data.text if data is not None else None)
    return vals

# En muchos exportes:
# - rows[0] puede ser un título
# - rows[1] suele ser la cabecera real con columnas
header = row_values(rows[1])
data = []

for r in rows[2:]:
    vals = row_values(r)
    if not vals:
        continue
    if len(vals) < len(header):
        vals += [None] * (len(header) - len(vals))
    data.append(vals[:len(header)])

df_raw = pd.DataFrame(data, columns=header)
print("Columnas leídas:", list(df_raw.columns))
df_raw.head(3)

# ---------------------------------------------------------
# 4) PASO A FORMATO "LARGO" (long): fecha + Q (m3/s)
#    Meses en año hidrológico (Oct..Sep). Convertimos a fechas reales.
# ---------------------------------------------------------
month_map = {"Oct":10, "Nov":11, "Dic":12, "Ene":1, "Feb":2, "Mar":3, "Abr":4, "May":5, "Jun":6, "Jul":7, "Ago":8, "Sep":9}

# Aseguramos que existen estas columnas; si alguna no está, revisa nombres en df_raw.columns
value_cols = [m for m in month_map.keys() if m in df_raw.columns]

df_long = df_raw.melt(
    id_vars=["Estación", "Año", "Día"],
    value_vars=value_cols,
    var_name="Mes",
    value_name="Q_m3s"
)

# Limpieza: "-" a NA, comas decimales, y conversión numérica
df_long["Q_m3s"] = df_long["Q_m3s"].replace("-", pd.NA)
df_long["Q_m3s"] = df_long["Q_m3s"].astype(str).str.replace(",", ".", regex=False)
df_long["Q_m3s"] = pd.to_numeric(df_long["Q_m3s"], errors="coerce")

# Año hidrológico "YYYY-YYYY"
y = df_long["Año"].astype(str).str.split("-", expand=True)
df_long["Y1"] = pd.to_numeric(y[0], errors="coerce")   # año de Oct-Dic
df_long["Y2"] = pd.to_numeric(y[1], errors="coerce")   # año de Ene-Sep

df_long["month"] = df_long["Mes"].map(month_map)
df_long["day"]   = pd.to_numeric(df_long["Día"], errors="coerce")

# Año calendario real: Oct-Nov-Dic -> Y1 ; Ene..Sep -> Y2
df_long["year"] = np.where(df_long["month"] >= 10, df_long["Y1"], df_long["Y2"])

df_long["date"] = pd.to_datetime(
    dict(year=df_long["year"], month=df_long["month"], day=df_long["day"]),
    errors="coerce"
)

df_daily = (
    df_long.dropna(subset=["date"])
           .sort_values("date")[["date", "Q_m3s"]]
           .reset_index(drop=True)
)

print("Rango:", df_daily["date"].min(), "->", df_daily["date"].max())
print("% datos no nulos:", df_daily["Q_m3s"].notna().mean())
df_daily.head(5), df_daily.tail(5)

# ---------------------------------------------------------
# 5) CONVERTIR A APORTACIONES TRIMESTRALES (hm3) CON CONTROL DE COMPLETITUD
#    IMPORTANTÍSIMO: si faltan muchos días, NO sumamos y ponemos NaN.
#    Trimestres hidrológicos:
#      T1: Oct-Dic, T2: Ene-Mar, T3: Abr-Jun, T4: Jul-Sep
# ---------------------------------------------------------
def hydro_quarter(month):
    if month in [10,11,12]:
        return 1
    elif month in [1,2,3]:
        return 2
    elif month in [4,5,6]:
        return 3
    else:
        return 4

# Año hidrológico: Oct-Dic pertenece al año hidrológico siguiente
df_daily["hydro_year"] = df_daily["date"].dt.year
df_daily.loc[df_daily["date"].dt.month >= 10, "hydro_year"] += 1

df_daily["hq"] = df_daily["date"].dt.month.apply(hydro_quarter)

# Volumen diario (m3) = Q(m3/s) * 86400
df_daily["V_m3"] = df_daily["Q_m3s"] * 86400

qt = (
    df_daily.groupby(["hydro_year","hq"])
    .agg(
        V_m3=("V_m3","sum"),
        n_valid=("Q_m3s","count"),
        n_total=("Q_m3s","size")
    )
    .reset_index()
)

qt["completitud"] = qt["n_valid"] / qt["n_total"]
qt["V_hm3"] = qt["V_m3"] / 1e6

# Criterio: trimestre válido si tiene al menos 90% de días con dato
qt.loc[qt["completitud"] < 0.90, "V_hm3"] = pd.NA

qt_clean = qt.dropna(subset=["V_hm3"]).sort_values(["hydro_year","hq"]).reset_index(drop=True)

print("Observaciones trimestrales válidas:", len(qt_clean))
print("Años hidrológicos:", qt_clean["hydro_year"].min(), "->", qt_clean["hydro_year"].max())
qt_clean.head(8)

# ---------------------------------------------------------
# 6) NORMALIZACIÓN POR TRIMESTRE (preparación del PAR)
#    En PAR(1) se suele trabajar sobre una serie aproximadamente normal.
#    Para empezar, usamos log-transform si hay asimetría fuerte.
#    (Más adelante haremos test de normalidad como exige el enunciado.)
# ---------------------------------------------------------
qt_clean["X"] = qt_clean["V_hm3"].astype(float)

# Transformación recomendada inicial (robusta en aportaciones):
qt_clean["Y"] = np.log(qt_clean["X"].clip(lower=1e-6))

# Normalización por trimestre: Z = (Y - mu_j) / sd_j
stats = qt_clean.groupby("hq")["Y"].agg(["mean","std"]).reset_index()
qt_clean = qt_clean.merge(stats, on="hq", how="left")
qt_clean["Z"] = (qt_clean["Y"] - qt_clean["mean"]) / qt_clean["std"]

qt_clean.head(8)

# ---------------------------------------------------------
# 7) AJUSTE PAR(1): estimación de phi_j y sigma_j por trimestre
#    Para cada trimestre j, ajustamos: Z_t = phi_j * Z_{t-1} + eps_t
#    donde la relación se toma ENTRE el mismo trimestre en años consecutivos.
# ---------------------------------------------------------
phi = {}
sigma = {}

for j in [1,2,3,4]:
    s = qt_clean[qt_clean["hq"] == j].sort_values("hydro_year").set_index("hydro_year")["Z"]
    # Valores de Z del mismo trimestre con un retardo de 1 año
    x = s.shift(1).dropna()
    y = s.loc[x.index]

    # Estimación OLS: phi = sum(x*y)/sum(x^2)
    xx = x.values
    yy = y.values
    phi_j = (xx @ yy) / (xx @ xx)

    resid = yy - phi_j * xx
    phi[j] = float(phi_j)
    sigma[j] = float(resid.std(ddof=1))

print("phi por trimestre hidrológico:", phi)
print("sigma (desv. típ. de residuo) por trimestre:", sigma)

# ---------------------------------------------------------
# 8) VERIFICACIÓN DE ESTACIONARIEDAD PERIÓDICA (PAR(1))
#    Criterio habitual: |prod(phi_j)| < 1
# ---------------------------------------------------------
prod_phi = math.prod([phi[j] for j in [1,2,3,4]])
print("Producto de phi_j:", prod_phi)
print("¿Cumple estacionariedad periódica (|prod|<1)?", abs(prod_phi) < 1)

# ---------------------------------------------------------
# 9) RESUMEN LISTO PARA PASAR A SIMULACIÓN (100 SERIES)
#    En este punto ya tenemos:
#      - serie trimestral hidrológica limpia (qt_clean)
#      - parámetros PAR(1): phi_j y sigma_j
#    Siguiente paso (cuando me digas): generar 100 series sintéticas
# ---------------------------------------------------------
qt_clean.tail(8)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Columnas leídas: ['Estación', 'Año', 'Día', 'Oct', 'Nov', 'Dic', 'Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep']
Rango: 1969-10-01 00:00:00 -> 2021-09-30 00:00:00
% datos no nulos: 0.9600042948408225
Observaciones trimestrales válidas: 190
Años hidrológicos: 1970 -> 2021
phi por trimestre hidrológico: {1: 0.3353661639198814, 2: 0.3996405378165834, 3: 0.3843046016033898, 4: 0.5384129099952824}
sigma (desv. típ. de residuo) por trimestre: {1: 0.9396383415944095, 2: 0.9204895284231169, 3: 0.9035953403375273, 4: 0.841490539060458}
Producto de phi_j: 0.027731912896652108
¿Cumple estacionariedad periódica (|prod|<1)? True


,hydro_year,hq,V_m3,n_valid,n_total,completitud,V_hm3,X,Y,mean,std,Z
182,2020,1,32807808.0,91,92,0.98913,32.807808,32.807808,3.490667,3.218967,0.921997,0.294686
183,2020,2,75645792.0,91,91,1.00000,75.645792,75.645792,4.326062,3.739051,0.880109,0.666976
184,2020,3,123247872.0,91,91,1.00000,123.247872,123.247872,4.814198,2.750636,1.674737,1.232171
185,2020,4,20043936.0,92,92,1.00000,20.043936,20.043936,2.997927,1.666754,1.645717,0.808871
186,2021,1,79411104.0,92,92,1.00000,79.411104,79.411104,4.374638,3.218967,0.921997,1.253443
187,2021,2,106921728.0,90,90,1.00000,106.921728,106.921728,4.672097,3.739051,0.880109,1.060149
188,2021,3,31726944.0,91,91,1.00000,31.726944,31.726944,3.457166,2.750636,1.674737,0.421875
189,2021,4,22444128.0,92,92,1.00000,22.444128,22.444128,3.111029,1.666754,1.645717,0.877596


In [2]:
# =========================================================
# 10) SIMULACIÓN DE 100 SERIES SINTÉTICAS PAR(1)
# =========================================================

np.random.seed(42)  # reproducibilidad

n_sim = 100
years = qt_clean["hydro_year"].unique()
n_years = len(years)

# Diccionarios con medias y desviaciones por trimestre
mu = dict(zip(stats["hq"], stats["mean"]))
sd = dict(zip(stats["hq"], stats["std"]))

# Estructura para almacenar simulaciones
simulations = []

for s in range(n_sim):
    sim_data = []

    # Inicialización: tomamos un valor inicial ~ N(0,1) por trimestre
    Z_prev = {j: np.random.normal(0, 1) for j in [1,2,3,4]}

    for y in years:
        for j in [1,2,3,4]:
            eps = np.random.normal(0, sigma[j])
            Z_new = phi[j] * Z_prev[j] + eps

            # Desnormalizar
            Y_new = mu[j] + sd[j] * Z_new
            X_new = np.exp(Y_new)   # revertimos log

            sim_data.append({
                "sim": s,
                "hydro_year": y,
                "hq": j,
                "V_hm3": X_new
            })

            Z_prev[j] = Z_new

    simulations.append(pd.DataFrame(sim_data))

df_sim = pd.concat(simulations, ignore_index=True)

print("Simulaciones generadas:", df_sim["sim"].nunique())
df_sim.head()


Simulaciones generadas: 100


,sim,hydro_year,hq,V_hm3
0,0,1970,1,23.800270
1,0,1970,2,33.139893
2,0,1970,3,259.120173
3,0,1970,4,59.089868
4,0,1971,1,16.374368


In [3]:
df_sim.groupby("hq")["V_hm3"].describe()


,count,mean,std,min,25%,50%,75%,max
hq,,,,,,,,
1,5100.0,39.586348,46.012901,0.766005,13.731185,25.711956,47.891865,753.886808
2,5100.0,61.906561,67.796004,2.097806,23.117972,42.025871,76.030803,1190.408669
3,5100.0,58.474016,222.904176,0.024082,5.099729,14.640270,45.631816,10967.524812
4,5100.0,22.567394,108.594845,0.008644,1.885641,5.595239,16.641310,3598.851781


In [4]:
# Percentil máximo histórico por trimestre
pmax = (
    qt_clean.groupby("hq")["V_hm3"]
    .quantile(0.995)
    .to_dict()
)

df_sim["V_hm3_trunc"] = df_sim.apply(
    lambda r: min(r["V_hm3"], pmax[r["hq"]]),
    axis=1
)


In [5]:
# =========================================================
# 11) TRUNCAMIENTO HIDROLÓGICAMENTE PLAUSIBLE (P99.5 HISTÓRICO)
#     Motivo: evitar extremos no plausibles derivados de la cola lognormal
# =========================================================

# Percentil 99.5 histórico por trimestre hidrológico
pmax_995 = (
    qt_clean.groupby("hq")["V_hm3"]
    .quantile(0.995)
    .to_dict()
)

print("P99.5 histórico por trimestre (hm3):", pmax_995)

# Aplicamos truncamiento superior a las series sintéticas
df_sim["V_hm3_trunc"] = df_sim.apply(
    lambda r: min(r["V_hm3"], pmax_995[r["hq"]]),
    axis=1
)

# Sanity check rápido tras truncamiento
df_sim.groupby("hq")["V_hm3_trunc"].describe()


P99.5 histórico por trimestre (hm3): {1: 116.61842592000005, 2: 161.09514144000002, 3: 211.87853856000004, 4: 88.42893120000002}


,count,mean,std,min,25%,50%,75%,max
hq,,,,,,,,
1,5100.0,36.105439,30.479353,0.766005,13.731185,25.711956,47.891865,116.618426
2,5100.0,56.018771,43.392852,2.097806,23.117972,42.025871,76.030803,161.095141
3,5100.0,39.169883,56.055043,0.024082,5.099729,14.640270,45.631816,211.878539
4,5100.0,14.732512,21.785364,0.008644,1.885641,5.595239,16.641310,88.428931


In [6]:
# =========================================================
# 12) FUNCIÓN GRINGORTEN PARA EXCEDENCIA
# =========================================================

def gringorten_exceedance(values):
    """
    Devuelve DataFrame con:
      - valor ordenado (descendente)
      - Pe: probabilidad empírica de excedencia (Gringorten)
    """
    v = pd.Series(values).dropna().sort_values(ascending=False).reset_index(drop=True)
    n = len(v)
    i = np.arange(1, n+1)  # 1..n
    Pe = (i - 0.44) / (n + 0.12)
    return pd.DataFrame({"value": v.values, "Pe": Pe})

def q_at_exceedance(values, Pe_target=0.75):
    """
    Devuelve el valor asociado a una excedencia objetivo (Pe_target),
    mediante interpolación lineal sobre la curva Gringorten.
    """
    g = gringorten_exceedance(values)
    # Pe crece con i; value decrece. Interpolamos value(Pe).
    # Para interpolar, necesitamos Pe ascendente:
    g2 = g.sort_values("Pe")
    return float(np.interp(Pe_target, g2["Pe"].values, g2["value"].values))


In [7]:
# =========================================================
# 13) UMBRAL HISTÓRICO (Pe = 0.75) POR TRIMESTRE
# =========================================================

hist_q75 = {}
for j in [1,2,3,4]:
    vals = qt_clean.loc[qt_clean["hq"]==j, "V_hm3"].astype(float).values
    hist_q75[j] = q_at_exceedance(vals, Pe_target=0.75)

hist_q75


{1: 18.630950400000003,
 2: 20.769739200000032,
 3: 4.508179200000001,
 4: 1.5848265600000002}

In [8]:
# =========================================================
# 14) UMBRAL SINTÉTICO (Pe = 0.75) POR TRIMESTRE Y SIMULACIÓN
# =========================================================

rows = []
for s in sorted(df_sim["sim"].unique()):
    for j in [1,2,3,4]:
        vals = df_sim.loc[(df_sim["sim"]==s) & (df_sim["hq"]==j), "V_hm3_trunc"].astype(float).values
        rows.append({"sim": s, "hq": j, "Q75": q_at_exceedance(vals, Pe_target=0.75)})

sim_q75 = pd.DataFrame(rows)

sim_q75.head(), sim_q75.groupby("hq")["Q75"].describe()


(   sim  hq        Q75
 0    0   1  12.583371
 1    0   2  30.269535
 2    0   3   3.142132
 3    0   4   2.037817
 4    1   1  12.322447,
     count       mean       std        min        25%        50%        75%  \
 hq                                                                           
 1   100.0  14.468377  3.509306   7.704946  12.127951  13.645294  15.978202   
 2   100.0  23.868457  4.966885  10.723710  20.448310  23.401450  27.634141   
 3   100.0   5.511881  2.003592   2.086684   4.205377   5.242017   6.532848   
 4   100.0   2.153598  1.146427   0.465359   1.379779   2.023564   2.631168   
 
           max  
 hq             
 1   27.098198  
 2   34.453040  
 3   11.579915  
 4    8.904581  )

In [9]:
# =========================================================
# 15) COMPARACIÓN HISTÓRICO vs SINTÉTICAS (Q75)
# =========================================================

summary = (
    sim_q75.groupby("hq")["Q75"]
    .agg([
        ("Q75_sint_media", "mean"),
        ("Q75_sint_p05", lambda x: np.quantile(x, 0.05)),
        ("Q75_sint_p95", lambda x: np.quantile(x, 0.95)),
        ("Q75_sint_min", "min"),
        ("Q75_sint_max", "max"),
    ])
    .reset_index()
)

summary["Q75_hist"] = summary["hq"].map(hist_q75)

# Reordenamos columnas para lectura
summary = summary[["hq","Q75_hist","Q75_sint_media","Q75_sint_p05","Q75_sint_p95","Q75_sint_min","Q75_sint_max"]]
summary


,hq,Q75_hist,Q75_sint_media,Q75_sint_p05,Q75_sint_p95,Q75_sint_min,Q75_sint_max
0,1,18.630950,14.468377,10.356198,21.585873,7.704946,27.098198
1,2,20.769739,23.868457,15.815459,31.691897,10.723710,34.453040
2,3,4.508179,5.511881,2.623904,9.614216,2.086684,11.579915
3,4,1.584827,2.153598,0.830785,3.991539,0.465359,8.904581


In [14]:
# =========================================================
# 16) GRÁFICOS CDF EMPÍRICAS (HISTÓRICA vs SINTÉTICAS) POR TRIMESTRE
# =========================================================
import matplotlib.pyplot as plt

for j in [1,2,3,4]:
    plt.figure()

    vals_h = qt_clean.loc[qt_clean["hq"]==j, "V_hm3"].astype(float).values
    xh, Fh = empirical_cdf(vals_h)
    plt.plot(xh, Fh, label="Histórica")

    ps = np.linspace(0.01, 0.99, 99)
    q_sims = []
    for s in sorted(df_sim["sim"].unique()):
        vals_s = df_sim.loc[(df_sim["sim"]==s) & (df_sim["hq"]==j), "V_hm3_trunc"].astype(float).values
        q_sims.append(np.quantile(vals_s, ps))
    q_sims = np.vstack(q_sims)

    q_p05 = np.quantile(q_sims, 0.05, axis=0)
    q_p95 = np.quantile(q_sims, 0.95, axis=0)

    plt.plot(q_p05, ps, label="Sintéticas P05")
    plt.plot(q_p95, ps, label="Sintéticas P95")

    plt.xlabel("Aportación trimestral (hm³)")
    plt.ylabel("F(x)")
    plt.title(f"CDF empírica – Trimestre hidrológico {j}")
    plt.legend()
    plt.tight_layout()

    fig_path = f"{base_out}/figures/CDF_Trimestre_{j}.png"
    plt.savefig(fig_path, dpi=300)
    plt.close()

    print("Guardada:", fig_path)



Guardada: /content/drive/MyDrive/PAR_BloqueII/outputs/figures/CDF_Trimestre_1.png
Guardada: /content/drive/MyDrive/PAR_BloqueII/outputs/figures/CDF_Trimestre_2.png
Guardada: /content/drive/MyDrive/PAR_BloqueII/outputs/figures/CDF_Trimestre_3.png
Guardada: /content/drive/MyDrive/PAR_BloqueII/outputs/figures/CDF_Trimestre_4.png


Crear carpetas (una sola vez)

In [11]:
import os

base_out = "BASE_DIR"
os.makedirs(base_out + "/tables", exist_ok=True)
os.makedirs(base_out + "/figures", exist_ok=True)


Guardar la tabla summary

In [12]:
summary_path = base_out + "/tables/Q75_comparacion_historico_sinteticas.csv"
summary.to_csv(summary_path, index=False)
summary_path


'/content/drive/MyDrive/PAR_BloqueII/outputs/tables/Q75_comparacion_historico_sinteticas.csv'

Excel (opcional, muy recomendable)

In [13]:
summary_xlsx = base_out + "/tables/Q75_comparacion_historico_sinteticas.xlsx"
summary.to_excel(summary_xlsx, index=False)
summary_xlsx


'/content/drive/MyDrive/PAR_BloqueII/outputs/tables/Q75_comparacion_historico_sinteticas.xlsx'

In [15]:
import os

base = "BASE_DIR"
os.makedirs(base + "/data_clean", exist_ok=True)

# 1) Serie diaria (fecha, Q)
df_daily_path = base + "/data_clean/aforo_9087_grisen_diario.csv"
df_daily.to_csv(df_daily_path, index=False)

# 2) Serie trimestral hidrológica limpia (hm3) + metadatos de completitud
qt_path = base + "/data_clean/aforo_9087_grisen_trimestral_hidrologico.csv"
qt_clean.to_csv(qt_path, index=False)

# 3) Serie trimestral con transformación y normalización (útil para anexos)
qt_norm_path = base + "/data_clean/aforo_9087_grisen_trimestral_normalizada.csv"
qt_clean.to_csv(qt_norm_path, index=False)

print("Guardados en data_clean:")
print(df_daily_path)
print(qt_path)
print(qt_norm_path)


Guardados en data_clean:
/content/drive/MyDrive/PAR_BloqueII/data_clean/aforo_9087_grisen_diario.csv
/content/drive/MyDrive/PAR_BloqueII/data_clean/aforo_9087_grisen_trimestral_hidrologico.csv
/content/drive/MyDrive/PAR_BloqueII/data_clean/aforo_9087_grisen_trimestral_normalizada.csv


In [16]:
qt_only = qt_clean[["hydro_year","hq","V_hm3"]].copy()
qt_only_path = base + "/data_clean/aforo_9087_grisen_trimestral_hm3.csv"
qt_only.to_csv(qt_only_path, index=False)
print("Guardado:", qt_only_path)


Guardado: /content/drive/MyDrive/PAR_BloqueII/data_clean/aforo_9087_grisen_trimestral_hm3.csv


Test de normalidad (sesgo obligatorio + Shapiro opcional)

2.1 Test de sesgo (obligatorio)

In [17]:
from scipy.stats import skew, shapiro
import numpy as np
import pandas as pd

normality_rows = []

for j in [1,2,3,4]:
    z = qt_clean.loc[qt_clean["hq"]==j, "Z"].astype(float).dropna().values
    n = len(z)

    g1 = skew(z, bias=False)  # asimetría muestral
    se = np.sqrt(6/n)         # error estándar aproximado del sesgo bajo normalidad
    z_skew = g1 / se          # estadístico tipo Z

    normality_rows.append({
        "hq": j,
        "n": n,
        "skew_g1": g1,
        "SE_skew": se,
        "Z_skew": z_skew
    })

normality_sesgo = pd.DataFrame(normality_rows)
normality_sesgo


,hq,n,skew_g1,SE_skew,Z_skew
0,1,47,-1.043443,0.357295,-2.920399
1,2,48,-0.875631,0.353553,-2.476659
2,3,47,-0.542725,0.357295,-1.518983
3,4,48,0.009147,0.353553,0.025872


2.2 Shapiro–Wilk (opcional pero recomendable)

In [18]:
shapiro_rows = []
for j in [1,2,3,4]:
    z = qt_clean.loc[qt_clean["hq"]==j, "Z"].astype(float).dropna().values
    W, p = shapiro(z)
    shapiro_rows.append({"hq": j, "W": W, "p_value": p})

normality_shapiro = pd.DataFrame(shapiro_rows)
normality_shapiro


,hq,W,p_value
0,1,0.928460,0.006665
1,2,0.940141,0.016433
2,3,0.945174,0.028106
3,4,0.956170,0.070691


In [19]:
out_tables = "BASE_DIR"
os.makedirs(out_tables, exist_ok=True)

normality_sesgo.to_csv(out_tables + "/normalidad_test_sesgo.csv", index=False)
normality_shapiro.to_csv(out_tables + "/normalidad_shapiro.csv", index=False)

print("Guardado normalidad en:", out_tables)


Guardado normalidad en: /content/drive/MyDrive/PAR_BloqueII/outputs/tables


In [20]:
# =========================================================
# 17) APLICACIÓN HIDROELÉCTRICA (ENERGÍA TRIMESTRAL)
# =========================================================

# Supuestos
rho = 1000        # kg/m3
g = 9.81          # m/s2
eta = 0.85        # rendimiento global
H = 20            # salto bruto (m)

energy_rows = []

for j in [1,2,3,4]:
    V_hm3 = hist_q75[j]                 # hm3 trimestral
    V_m3 = V_hm3 * 1e6                  # m3
    E_J = rho * g * H * eta * V_m3      # energía en julios
    E_GWh = E_J / 3.6e12                # conversión a GWh

    energy_rows.append({
        "Trimestre_hidrologico": j,
        "Q75_hm3": V_hm3,
        "Salto_m": H,
        "Rendimiento": eta,
        "Energia_GWh": E_GWh
    })

energy_df = pd.DataFrame(energy_rows)
energy_df


,Trimestre_hidrologico,Q75_hm3,Salto_m,Rendimiento,Energia_GWh
0,1,18.630950,20,0.85,0.863079
1,2,20.769739,20,0.85,0.962158
2,3,4.508179,20,0.85,0.208841
3,4,1.584827,20,0.85,0.073417


Guardar tabla de energía en Drive (CSV + Excel)

In [21]:
import os

base_out = "BASE_DIR"
os.makedirs(base_out + "/tables", exist_ok=True)

energy_csv = base_out + "/tables/energia_Q75_hidroelectrica.csv"
energy_xlsx = base_out + "/tables/energia_Q75_hidroelectrica.xlsx"

energy_df.to_csv(energy_csv, index=False)
energy_df.to_excel(energy_xlsx, index=False)

print("Guardado:", energy_csv)
print("Guardado:", energy_xlsx)


Guardado: /content/drive/MyDrive/PAR_BloqueII/outputs/tables/energia_Q75_hidroelectrica.csv
Guardado: /content/drive/MyDrive/PAR_BloqueII/outputs/tables/energia_Q75_hidroelectrica.xlsx


Guardar tabla resumen “lista para pegar” en el informe

In [22]:
energy_df_fmt = energy_df.copy()
energy_df_fmt["Q75_hm3"] = energy_df_fmt["Q75_hm3"].round(3)
energy_df_fmt["Energia_GWh"] = energy_df_fmt["Energia_GWh"].round(3)

energy_fmt_csv = base_out + "/tables/energia_Q75_hidroelectrica_redondeada.csv"
energy_df_fmt.to_csv(energy_fmt_csv, index=False)

print("Guardado:", energy_fmt_csv)
energy_df_fmt


Guardado: /content/drive/MyDrive/PAR_BloqueII/outputs/tables/energia_Q75_hidroelectrica_redondeada.csv


,Trimestre_hidrologico,Q75_hm3,Salto_m,Rendimiento,Energia_GWh
0,1,18.631,20,0.85,0.863
1,2,20.770,20,0.85,0.962
2,3,4.508,20,0.85,0.209
3,4,1.585,20,0.85,0.073


## Export

Run the following cell to export key results to Excel in your Drive.

In [ ]:
# =========================================================
# Export outputs to Drive (tables + figures)
# =========================================================
import os

# Tables expected: summary, energy_df, normality_sesgo, normality_shapiro, params_df, wide (compact data)
exports = [
    ("Q75_comparacion_historico_sinteticas.xlsx", "summary"),
    ("energia_Q75_hidroelectrica.xlsx", "energy_df"),
    ("normalidad_test_sesgo.xlsx", "normality_sesgo"),
    ("normalidad_shapiro.xlsx", "normality_shapiro"),
    ("PAR_parametros.xlsx", "params_df"),
    ("aforo_trimestral_compacto.xlsx", "wide"),
]

for fname, var in exports:
    if var in globals():
        path = os.path.join(TAB_DIR, fname)
        globals()[var].to_excel(path, index=False)
        print("Saved:", path)
    else:
        print("Skipped (variable not found):", var)

# Figures: save any matplotlib figures already created in the run (recommended: use savefig() inline)
print("Tables folder:", TAB_DIR)
print("Figures folder:", FIG_DIR)
